In [1]:
import classiq
classiq.authenticate()

/opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:101: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
import torch
import torch.optim as optim
import tqdm
import math
from classiq import *
from classiq import (
    synthesize,
    qfunc,
    QArray,
    QBit,
    RX,
    CArray,
    Output,
    CReal,
    repeat,
    create_model,
    show
)
from classiq.execution import execute_qnn
from classiq.applications.qnn import QLayer
from classiq.qmod.symbolic import pi
from torch.nn.utils.rnn import pad_sequence
import torchvision.transforms as transforms
from torchvision import datasets
from classiq.execution import (
    ExecutionPreferences,
    execute_qnn,
    set_quantum_program_execution_preferences,
)
from classiq.synthesis import SerializedQuantumProgram
from classiq.applications.qnn.types import (
    MultipleArguments,
    ResultsCollection,
    SavedResult,
)

/opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <CFB4297E-118A-3D27-BF39-55106327812A> /opt/anaconda3/envs/baler/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
N_QUBITS = 4
num_shots = 1000

In [4]:
def execute(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    quantum_program = set_quantum_program_execution_preferences(
        quantum_program, preferences=ExecutionPreferences(num_shots=num_shots)
    )
    return execute_qnn(quantum_program, arguments)

In [5]:
def post_process(result: SavedResult) -> torch.Tensor:
    res = result.value
    yvec = [
        (res.counts_of_qubits(k)["1"] if "1" in res.counts_of_qubits(k) else 0)
        / num_shots
        for k in range(N_QUBITS)
    ]

    return torch.tensor(yvec)

##### Quantum Vision Transformer:
$$
  y = f_{3} \circ f_{2} \circ f_{1} \circ	 f_{0} \circ(X)
$$
, where X - input tensor, y - result, $$f_{i}$$ is the Neural Network Layer


In [6]:
def get_circuit():


    @qfunc
    def vqc(weight_: CArray[CArray[CReal, N_QUBITS], N_QUBITS], res:QArray) -> None:
        
        num_qubits = N_QUBITS
        num_qlayers = N_QUBITS
        
        repeat(
            count=num_qlayers,
            iteration=lambda i: repeat(count=num_qubits,  iteration=lambda j: RX(pi * weight_[i][j], res[j]))
        )
        
        repeat(
            count=num_qubits - 1,
            iteration=lambda index: CX(ctrl=res[index], target=res[index + 1]),
        )

    
    @qfunc
    def main(input_: CArray[CReal, N_QUBITS], weight_: CArray[CArray[CReal, N_QUBITS], N_QUBITS], res: Output[QArray[QBit, N_QUBITS]]) -> None:
        

        encode_in_angle(input_, res)
        vqc(weight_, res)


    qmod = create_model(main)
    quantum_program  = synthesize(qmod)
    return quantum_program


In [7]:
from IPython.display import Image 
  
# get the image 
Image(url="axioms-13-00323-g004-550.jpg", width=800, height=400) 


![title]("axioms-13-00323-g004-550.jpg")

In [61]:
Image(url="axioms-13-00323-g005-550.jpg", width=800, height=300) 


In [62]:
class Patchify(torch.nn.Module):
    """
    Patchify layer implemented using the Conv2d layer
    """
    def __init__(self, in_channels:int, patch_size:int, hidden_size:int):
        super(Patchify, self).__init__()
        self.patch_size = patch_size
        self.conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=hidden_size, kernel_size=self.patch_size, stride=self.patch_size)
        self.hidden_size = hidden_size
        
    def forward(self, x:torch.Tensor):
        bs, c, h, w = x.size()
        self.num_patches = (h // self.patch_size) ** 2

        x = self.conv(x)
        x = x.view(bs, self.num_patches, self.hidden_size)
        return x

##### Quantum Vision Transformer:
$$
  y = f_{3} \circ f_{2} \circ f_{1} \circ	 f_{0} \circ(X)
$$
, where X - input tensor, y - result, $$f_{i}$$ is the Neural Network Layer


In [ ]:
class RotaryPositionalEmbedding(torch.nn.Module):
    """
    Rotary Positional Embedding
    """
    def __init__(self, d_model, max_seq_len):
        super(RotaryPositionalEmbedding, self).__init__()

        # Create a rotation matrix.
        self.rotation_matrix = torch.zeros(d_model, d_model)
        for i in range(d_model):
            for j in range(d_model):
                self.rotation_matrix[i, j] = math.cos(i * j * 0.01)

        # Create a positional embedding matrix.
        self.positional_embedding = torch.zeros(max_seq_len, d_model)
        for i in range(max_seq_len):
            for j in range(d_model):
                self.positional_embedding[i, j] = math.cos(i * j * 0.01)

    def forward(self, x):
        """
        Args:
            x: A tensor of shape (batch_size, seq_len, d_model).

        Returns:
            A tensor of shape (batch_size, seq_len, d_model).
        """

        # Add the positional embedding to the input tensor.
        x += self.positional_embedding

        # Apply the rotation matrix to the input tensor.
        x = torch.matmul(x, self.rotation_matrix)

        return x

In [ ]:
quantum_program = get_circuit()
show(quantum_program)

Quantum program link: https://platform.classiq.io/circuit/2we1LvLdzd9RQ9vSPLE1T3WcIxn?login=True&version=0.77.0


In [ ]:
class QuantumLayer(torch.nn.Module):
    """
    Quantum Layer
    """
    def __init__(self, in_dim, out_dim):
        super(QuantumLayer, self).__init__()
        self.quantum_program = get_circuit()
        self.quantum_layer = QLayer(quantum_program, execute_qnn, post_process)

    def forward(self, x:torch.Tensor):
        x = self.quantum_layer(x)
        return x

##### Feed Forward Neural Network:
$$
  y = f_{3} \circ f_{2} \circ f_{1} \circ	 f_{0} \circ(X)
$$
, where X - input tensor, y - result, $$f_{i}$$ is the Neural Network Layer


In [ ]:
class FFN(torch.nn.Module):
    """
    Feed Forward Network
    """
    def __init__(self, in_dim, hidden_size):
        super().__init__()
        self.linear_1 = torch.nn.Linear(in_dim, hidden_size)
        self.qlinear = QuantumLayer(hidden_size, hidden_size)
        self.dropout = torch.nn.Dropout(p=0.4)
        self.linear_2 = torch.nn.Linear(hidden_size, in_dim)
        return
    
    def forward(self, x:torch.Tensor):
        seq_len = x.size()[1]
        x = self.linear_1(x)
        x = [self.qlinear(x[:, t, :]) for t in range(seq_len)]
        x = torch.Tensor(pad_sequence(x))
        x = self.dropout(x)
        x = torch.nn.functional.gelu(x)
        x = self.linear_2(x)
        return x

##### Feed Forward Neural Network:
$$
  y = f_{3} \circ f_{2} \circ f_{1} \circ	 f_{0} \circ(X)
$$
, where X - input tensor, y - result, $$f_{i}$$ is the Neural Network Layer


In [ ]:
class qMHA(torch.nn.Module):
    """
    Quantum Multihead Attention
    """
    def __init__(self, in_dim:int, num_heads:int) -> None:
        super().__init__()

        self.k_linear = QuantumLayer(in_dim, in_dim);
        self.q_linear = QuantumLayer(in_dim, in_dim);
        self.v_linear = QuantumLayer(in_dim, in_dim);
        self.dropout = torch.nn.Dropout(p=0.1)
        
        self.final_l = QuantumLayer(in_dim, in_dim)
        self.num_heads = num_heads
        self.in_dim = in_dim
        
        return

    def forward(self, X:torch.Tensor):

        seq_len = X.size()[1]
        K = [self.k_linear(X[:, t, :]) for t in range(seq_len)]
        Q = [self.q_linear(X[:, t, :]) for t in range(seq_len)]
        V = [self.v_linear(X[:, t, :]) for t in range(seq_len)]
        
        k = torch.Tensor(pad_sequence(K))
        q = torch.Tensor(pad_sequence(Q))
        v = torch.Tensor(pad_sequence(V))
    
        attention = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        attention = torch.nn.functional.softmax(attention, dim=-1)

        attention = self.dropout(attention)
        attention = attention @ v 
        #x = self.final_l(attention)
        return attention

In [ ]:
class qTransformerEncoder(torch.nn.Module):
    """
    Quantum Transformer Encoder Layer
    """
    def __init__(self, in_dim:int, num_heads:int) -> None:
        super().__init__()
        
        self.layer_norm_1 = torch.nn.LayerNorm(normalized_shape=in_dim)
        self.layer_norm_2 = torch.nn.LayerNorm(normalized_shape=in_dim)
        
        self.qMHA = qMHA(in_dim, num_heads)
        self.qFFN = FFN(in_dim, hidden_size=in_dim)
        self.dropout = torch.nn.Dropout(p=0.3)
        

    def forward(self, X:torch.Tensor):
        x = self.qMHA(X)
        x = (self.layer_norm_1(x) + X)
        x = self.dropout(x)
        
        y = self.qFFN(x)
        y = self.layer_norm_2(y)+x
        return y

##### Feed Forward Neural Network:
$$
  y = f_{3} \circ f_{2} \circ f_{1} \circ	 f_{0} \circ(X)
$$
, where X - input tensor, y - result, $$f_{i}$$ is the Neural Network Layer


In [ ]:
class QVT(torch.nn.Module):
    """
    Quantum Vision Transformer;
    """
    def __init__(self, in_channels, patch_size, in_dim, hidden_size,  num_heads, n_classes, n_layers) -> None:
        super().__init__()
        
        self.d_model = (in_dim//patch_size)**2
        self.n_classes = n_classes

        self.patch_formation = Patchify(in_channels=in_channels, patch_size=patch_size, hidden_size=hidden_size)

        self.pos_encoding = RotaryPositionalEmbedding(hidden_size, self.d_model)
        self.transformer_blocks = torch.nn.ModuleList([qTransformerEncoder(hidden_size, num_heads) for i in range(n_layers)])
                
        self.final_normalization = torch.nn.LayerNorm(hidden_size)
        self.final_layer = torch.nn.Linear(hidden_size, self.n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:  
        
        x = self.patch_formation(x)
        x += self.pos_encoding(x)
        
        for trans_block in self.transformer_blocks:
            x = trans_block(x)
        
        x = self.final_normalization(x)
        x = x.mean(axis=1)
        x = self.final_layer(x)
        
        return x

In [ ]:
#### Example with the MNIST Dataset:
transform=transforms.Compose([
                          transforms.ToTensor(), # first, convert image to PyTorch tensor
                          transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                      ])
dataset1 = datasets.MNIST('../data', train=True, download=True,transform=transform)
dataset2 = datasets.MNIST('../data', train=False,transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1,batch_size=16)
test_loader = torch.utils.data.DataLoader(dataset2,batch_size=1)

In [68]:

clf = QVT(in_channels=1, patch_size=7, in_dim=28, hidden_size=4, num_heads=1, n_classes=10, n_layers=2)

opt = optim.SGD(clf.parameters(), lr=0.01, momentum=0.5)

loss_history = []
acc_history = []

In [69]:
for data, label in tqdm.tqdm(train_loader):
    # forward pass, calculate loss and backprop!
    opt.zero_grad()

    preds = clf(data)
    
    loss = torch.nn.functional.nll_loss(preds, label)
    loss.backward()
    try:
        loss_history.append(loss)
        print(loss)
    except Exception as e:
        print(e)
    opt.step()

  0%|          | 0/3750 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-886' coro=<ExecutionJob.result_async() done, defined at /opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/execution/jobs.py:123> exception=RuntimeError('Cannot send a request, as the client has been closed.')>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/baler/lib/python3.11/asyncio/tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/execution/jobs.py", line 128, in result_async
    await self.poll_async(timeout_sec=timeout_sec, _http_client=_http_client)
  File "/opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/execution/jobs.py", line 253, in poll_async
    await self._poll_job(timeout_sec=timeout_sec, _http_client=_http_client)
  File "/opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/execution/jobs.py", line 271,

ClassiqAPIError: Call to API failed with code 500: Internal error occurred. Please contact Classiq support.

Error identifier: E19D7F0AC-1EBC-4508-89F6-8D13FEABC06E
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack or open a support ticket at: https://classiq-community.freshdesk.com/support/tickets/new

In [ ]:


def train():
    clf.train() # set model in training mode (need this because of dropout)
    
    # dataset API gives us pythonic batching 
    for data, label in tqdm.tqdm(train_loader):
        # forward pass, calculate loss and backprop!
        opt.zero_grad()

        preds = clf(data)
        
        loss = torch.nn.functional.nll_loss(preds, label)
        loss.backward()
        try:
            loss_history.append(loss)
            print(loss)
        except Exception as e:
            print(e)
        opt.step()
    return loss_history

def test():
    clf.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for data, target in tqdm.tqdm(test_loader):
        
        output = clf(data)
        test_loss += torch.nn.functional.nll_loss(output, target).item()
        pred = output.argmax() # get the index of the max log-probability
        correct += pred.eq(target).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    accuracy = 100. * correct / len(test_loader.dataset)
    acc_history.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

In [ ]:
#for epoch in range(0, 3):
train()

  0%|          | 0/3750 [05:44<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.style.use('fivethirtyeight')
plt.title('Model Loss')
plt.plot(range(1, epochs+1), history.history['loss'], label="training")
plt.plot(range(1, epochs+1), history.history['val_loss'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()